In [1]:
import itertools
from pathlib import Path

import geopandas as gpd
import pandas as pd

from shapely.validation import make_valid
from shapely.prepared import prep
from shapely import strtree
from shapely.geometry import Point, MultiPolygon, Polygon

from tqdm import tqdm

### Set Paths

- ```_temp``` holds precomputed polygons, intermediate weight outputs, etc.
- ```weights``` will hold the final allocation weights.

In [50]:
data_dir_path = Path('.')

(data_dir_path / '_temp').mkdir(parents=False, exist_ok=True)
(data_dir_path / 'weights').mkdir(parents=False, exist_ok=True)

### Load in Target Geographies (Locale Data)

Create one polygon for each locale type for each state to simplify and (hopefully) speed up intersection computations.  This is time-consuming so we save the results and re-load as required.

In [3]:
locale_dict = {1: 'City', 2: 'Suburban', 3: 'Town', 4: 'Rural'}


def fill_polygon(geo):
    geo = MultiPolygon([Polygon(g) for g in geo.geoms])
    if not geo.is_valid:
        geo = make_valid(geo)
    return geo


if (data_dir_path / '_temp' / '_temp_state_level_locale_polygons.shp').exists():
    state_level_locales_dataframe = gpd.read_file(data_dir_path / '_temp' / '_temp_state_level_locale_polygons.shp')
    
    ## GeoPandas was only saving the boundaries of geometries in my environment,
    ## I'm not sure if this happens in all environments, but this corrects it
    state_level_locales_dataframe.geometry = \
    state_level_locales_dataframe.geometry.apply(lambda geo: fill_polygon(geo) if geo is not None else None)
    
else:
    locales_dataframe = gpd.read_file(data_dir_path / 'edge_locales' / 'edge_locale21_nces_all_us.shp')
    locations_dataframe = gpd.read_file(data_dir_path / 'edge_locations' / 'EDGE_GEOCODE_PUBLICSCH_2021.shp')

    ## Combine all sub-types of each locale into one
    locales_dataframe['localetype'] = locales_dataframe.LOCALE.apply(lambda l: locale_dict[int(l) // 10])

    state_level_locale_polygon_data = []

    for state in tqdm(locales_dataframe.STATEFP.unique()):
        locale_state_dateframe = locales_dataframe[locales_dataframe.STATEFP == state]

        for localetype in locale_dict.values():
            localetype_dataframe = locale_state_dateframe[locale_state_dateframe.localetype == localetype]

            intersected_geo = None
            for geo in localetype_dataframe.geometry:
                if not geo.is_valid:
                    locale_geo = make_valid(geo)
                else:
                    locale_geo = geo

                if intersected_geo is None:
                    intersected_geo = locale_geo
                else:
                    intersected_geo = intersected_geo.union(locale_geo)

            state_level_locale_polygon_data.append({'STATEFP': state,
                                                    'LOCALETYPE': localetype,
                                                    'geometry': intersected_geo})

    state_level_locales_dataframe = gpd.geodataframe.GeoDataFrame(state_level_locale_polygon_data)
    state_level_locales_dataframe.to_file(data_dir_path / '_temp' / '_temp_state_level_locale_polygons.shp')
    del locales_dataframe

### Load in Source Geographies

Currently includes states and ZIP codes (technically [ZIP code tabulation areas](https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html)).

Also load [FIPS state codes](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) to join dataframes on.

In [4]:
zipcodes_dataframe = gpd.read_file(data_dir_path / 'zip_codes' / 'tl_2022_us_zcta520.shp')
states_dataframe = gpd.read_file(data_dir_path / 'states' / 'tl_2022_us_state.shp')

fips_state_code_dataframe = pd.read_csv(data_dir_path / 'fips_state_codes.csv')

Add in a column for the state(s) that each ZIP code intersects, so we don't have to calculate intersections against every single ZIP code later.

In [22]:
zipcodes_dataframe['STATEFP'] = None
for zip_index, zip_row in tqdm(zipcodes_dataframe.iterrows(), total=len(zipcodes_dataframe)):
    states = set()
    prepared_zip_geometry = prep(zip_row.geometry)
    for _, state_row in states_dataframe.iterrows():
        if prepared_zip_geometry.intersects(state_row.geometry):
            states.add(state_row.GEOID)
    
    zipcodes_dataframe.at[zip_index, 'STATEFP'] = list(states)

100%|█████████████████████████████████████████████████████████████████████████████| 33791/33791 [02:04<00:00, 272.49it/s]


Define functions to load in census tract data (by state).

In [129]:
def safe_union(geo1, geo2):
    '''Utility function to union geographies
    when one or both may be "None".'''
    
    if geo1 is not None and geo2 is not None:
        return geo1.union(geo2)
    elif geo1 is None:
        return geo2
    elif geo2 is None:
        return geo1
    else:
        return None
    
    
def get_tracts_dataframe_and_strtree(state_fips_code):
    tract_path = data_dir_path / 'census_tracts' / 'tl_2022_{}_tract.zip'.format(state_fips_code)
    tracts_dataframe =  gpd.read_file(tract_path)
    
    ## Census tracts appear to cover out into oceans. Intersect each tract with the state-level locale 
    ## geography (which only covers up to the coastline) to ensure coastal tracts are not biased
    state_land_geo = state_level_locales_dataframe[state_level_locales_dataframe.STATEFP == state_fips_code].geometry
    state_land_geo = list(itertools.accumulate(state_land_geo, safe_union))[-1]
    
    tract_geos = [make_valid(geo) if not geo.is_valid else geo for geo in tracts_dataframe.geometry]
    tract_geos = [state_land_geo.intersection(geo) for geo in tract_geos]
    
    tracts_index = {id(geo): i for i, geo in enumerate(tract_geos)}
    tracts_strtree = strtree.STRtree(tract_geos)
    
    return tracts_dataframe, tracts_index, tracts_strtree


In [144]:
def calculate_allocations(state_level_locales_dataframe, 
                          source_geography_name, source_geography_id_column_name,
                          source_geography_dataframe, source_geography_strtree,
                          fips_state_code_dataframe=fips_state_code_dataframe,
                          data_dir_path=data_dir_path):
    
    source_geography_id_column = source_geography_dataframe[source_geography_id_column_name]
    
    ## Ensure we can map back to original row indices when querying the STRTree
    source_geography_index = {id(geo): i for i, geo in enumerate(source_geography_dataframe.geometry)}

    with tqdm(state_level_locales_dataframe.STATEFP.unique(), position=0, leave=False) as progress:
        for state in progress:
            state_string = fips_state_code_dataframe[fips_state_code_dataframe.STATEFP == int(state)].iloc[0].STATENAME

            tracts_dataframe, tracts_index, tracts_strtree = get_tracts_dataframe_and_strtree(state)
            
            for localetype in locale_dict.values():
                progress.set_postfix({'state': state_string, 'locale_type': localetype}, refresh=True)
                state_level_locale_geo = state_level_locales_dataframe[(state_level_locales_dataframe.STATEFP == state) & \
                                                                       (state_level_locales_dataframe.LOCALETYPE == localetype)].iloc[0].geometry
                ## Some states don't have some locales
                if state_level_locale_geo is None:
                    continue

                source_geography_weights = []
                
                ## The STRTree returns only potentially intersecting geometries - we iterate 
                ## through those, confirm they intersect, and record the allocation weights
                for source_geometry in source_geography_strtree.query(state_level_locale_geo):
                    if source_geometry.intersects(state_level_locale_geo):
                        inter = state_level_locale_geo.intersection(source_geometry)
                        inter_area = inter.area
                        
                        inter_area_target = round(inter_area / state_level_locale_geo.area, 5)
                        inter_area_source = round(inter_area / source_geometry.area, 5)
                        
                        if inter_area_target > 0: ## Ignore small, spurious intersections
                            
                            tract_ids = []
                            
                            ## Fraction of the tract taken up by the intersection 
                            ## of the locale area and source geography
                            tract_allocation_weights_tract = [] 
                            
                            ## Fraction of the intersection of the locale area 
                            ## and source geography taken up by the tract
                            tract_allocation_weights_inter = []

                            for tract_geo in tracts_strtree.query(inter):
                                if tract_geo.intersects(inter):
                                    tract_id = tracts_dataframe.GEOID.iloc[tracts_index[id(tract_geo)]]
                                    inter_tract = tract_geo.intersection(inter)
                                    inter_tract_area = inter_tract.area

                                    inter_tract_area_tract = round(inter_tract_area / tract_geo.area, 5)
                                    inter_tract_area_inter = round(inter_tract_area / inter.area, 5)

                                    if inter_tract_area_inter > 0:
                                        tract_ids.append(tract_id)
                                        tract_allocation_weights_tract.append(inter_tract_area_tract)
                                        tract_allocation_weights_inter.append(inter_tract_area_inter)

                            tract_alloc_data_str = ';'.join(['{0}:{1}:{2}'.format(trid, trwt, trwi) for trid, trwt, trwi in \
                                                             zip(tract_ids, tract_allocation_weights_tract, tract_allocation_weights_inter)])

                            source_geography_weights.append({source_geography_id_column_name: source_geography_id_column[source_geography_index[id(source_geometry)]],
                                                             'FRACAREA_TARGET': inter_area_target,
                                                             'FRACAREA_SOURCE': inter_area_source,
                                                             'FRACAREA_TRACTS': tract_alloc_data_str})  
                            
                        #return state_level_locale_geo, source_geometry
                    
                source_geography_weights = pd.DataFrame(source_geography_weights)
                source_geography_weights.to_csv(data_dir_path / '_temp' / '_temp_{}-{}_{}_weights.csv'.format(state, localetype, source_geography_name), index=False)
    
    ## Merge all temp files - note that these are
    ## NOT automatically deleted, just in case!
    combined_weights = []
    for weight_csv_path in (data_dir_path / '_temp').glob('*_{}_weights.csv'.format(source_geography_name)):
        weight_df = pd.read_csv(weight_csv_path)
        state_code, localetype = weight_csv_path.name[6:].split('_')[0].split('-')

        weight_df['STATEFP'] = state_code
        weight_df['localetype'] = localetype

        combined_weights.append(weight_df)

    weight_df = pd.concat(combined_weights, axis=0, ignore_index=True)
    weight_df.to_csv(data_dir_path / 'weights' / '{}_weights.csv'.format(source_geography_name), index=False)



In [145]:
state_strtree = strtree.STRtree([make_valid(geo) if not geo.is_valid else geo for geo in states_dataframe.geometry])

calculate_allocations(state_level_locales_dataframe,
                      'state', 'STATEFP',
                      states_dataframe, state_strtree)

C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\2669388252.py:1: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  state_strtree = strtree.STRtree([make_valid(geo) if not geo.is_valid else geo for geo in states_dataframe.geometry])
  0%|                                                                                             | 0/56 [00:00<?, ?it/s]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
  2%|▊                                           | 1/56 [00:00<00:44,  1.24it/s, state=AMERICAN SAMOA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
  4%|█▌                 

 41%|████████████████████▏                            | 23/56 [46:02<2:00:47, 219.64s/it, state=MAINE, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 43%|███████████████████▋                          | 24/56 [58:19<3:20:00, 375.02s/it, state=MICHIGAN, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 45%|██████████████████▎                      | 25/56 [58:38<2:18:33, 268.17s/it, state=NEW HAMPSHIRE, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = st

 84%|███████████████████████████████▉      | 47/56 [8:07:15<2:08:54, 859.36s/it, state=NORTH CAROLINA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 86%|████████████████████████████████▌     | 48/56 [8:15:21<1:39:39, 747.40s/it, state=SOUTH CAROLINA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 88%|██████████████████████████████████▏    | 49/56 [8:16:19<1:03:03, 540.48s/it, state=WEST VIRGINIA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = st

In [146]:
zipcode_strtree = strtree.STRtree([make_valid(geo) if not geo.is_valid else geo for geo in zipcodes_dataframe.geometry])

calculate_allocations(state_level_locales_dataframe,
                      'zipcode', 'ZCTA5CE20',
                      zipcodes_dataframe, zipcode_strtree)

C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\992972759.py:1: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  zipcode_strtree = strtree.STRtree([make_valid(geo) if not geo.is_valid else geo for geo in zipcodes_dataframe.geometry])
  0%|                                                                                             | 0/56 [00:00<?, ?it/s]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
  2%|▊                                           | 1/56 [00:02<02:01,  2.20s/it, state=AMERICAN SAMOA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
  4%|█▋              

 41%|████████████████████▏                            | 23/56 [37:12<1:43:12, 187.66s/it, state=MAINE, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 43%|███████████████████▋                          | 24/56 [44:57<2:24:26, 270.82s/it, state=MICHIGAN, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 45%|██████████████████▎                      | 25/56 [45:22<1:41:52, 197.18s/it, state=NEW HAMPSHIRE, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = st

 84%|███████████████████████████████▉      | 47/56 [3:53:00<1:06:21, 442.38s/it, state=NORTH CAROLINA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 86%|██████████████████████████████████▎     | 48/56 [3:57:39<52:26, 393.33s/it, state=SOUTH CAROLINA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = strtree.STRtree(tract_geos)
 88%|███████████████████████████████████▉     | 49/56 [4:00:45<38:37, 331.06s/it, state=WEST VIRGINIA, locale_type=Rural]C:\Users\drawi\AppData\Local\Temp\ipykernel_12492\1253621424.py:28: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tracts_strtree = st

----------------